## Midterm Project Using Kaggle Datasets

dataset url : https://www.kaggle.com/datasets/samuelcortinhas/muffin-vs-chihuahua-image-classification?select=test

### Task : Using Image Classification for Distinguishing Muffin and Chihuahua

* The goal of the project is to create a model that uses SVC and CNN supervised learning to distinguish between muffins and chihuahua.
* 이 프로젝트의 목표는 SVC와 CNN 지도학습을 이용하여 머핀과 치와와를 구분할 수 있는 모델을 만드는 것 입니다.

<img src="https://i.postimg.cc/2SXNWP7f/muffin-meme2.jpg"  width="30%" height="30%"></img>


### 데이터셋 설명
* 모든 데이터 셋은 muffin과 chihuahua 단어에 대해 구글 이미지 결과값을 스크랩한 바이너리 이미지 파일로 구성되어있습니다.

* 선택한 데이터 셋은 아래와 같은 두 가지의 데이터 셋으로 나뉘어있습니다.
    * train
    * test
    * 각각의 데이터 셋 내부에는 두 가지의 레이블이 존재합니다.
        * muffin
        * chihuahua
        
* 모델 학습 시 데이터 셋은 다음과 같이 분류하여 사용하였습니다.
    * train data : 학습 데이터 (3733장의 이미지)
    * validation data : 모델 성능 검증을 위한 데이터 (1000장의 이미지)
    * train data : 모델 학습 종료 후 모델을 테스트하기 위한 데이터 (1184장의 이미지)


### Dataset Description
* All datasets consist of binary image files obtained by scraping Google Image search results for the words "muffin" and "chihuahua."

* The selected dataset is divided into two main datasets:
    * Train
    * Test
    * Each dataset contains two labels:
        * Muffin
        * Chihuahua
* During model training, the datasets were categorized as follows:
    * Train Data: Used for model training (3733 images)
    * Validation Data: Used for model performance validation (1000 images)
    * Test Data: Used to test the model after training completion (1184 images)

### 모델 학습 및 테스트 환경

* 구글 Colab에서 T4GPU를 사용하여 모델 학습 및 테스트 진행.

### 이미지 분류를 위해 선택한 지도학습 알고리즘

1. SVC(Support Vector Machine with a Linear Kernel)
    * SVM은 주어진 데이터들을 분류하는 가장 최적의 결정 경계를 찾는 알고리즘입니다.
        * SVC는 SVM을 이용하여 분류 작업 시, 선형 커널을 적용한 지도학습 알고리즘입니다.
2. CNN (Convolutional Neural Network)
    * 입력된 이미지에 대해 필터(커널)를 사용하여 이미지의 특징을 추출하고, 학습하는 알고리즘입니다.
    * 이미지에 대해 먼저 커널과 합성곱(Convolutional Layers)이 이루어지고, 이후에는 Pooling Layer를 거쳐 가장 특징적인 정보를 추출합니다.
    * 특징 정보들은 다층 퍼셉트론(MLP)을 통해 분류 작업에 사용됩니다.

### Model Training and Testing Environment

* Model training and testing were conducted using T4 GPU on Google Colab.

### Supervised Learning Algorithms Chosen for Image Classification

1. SVC (Support Vector Machine with a Linear Kernel)
    * SVM is an algorithm that finds the optimal boundary for classifying given data.
        * SVC is a supervised learning algorithm that applies a linear kernel to classify data using SVM.
2. CNN (Convolutional Neural Network)
    * It is an algorithm that extracts and learns features of an image using filters (kernels).
    * For images, convolutional layers are applied to the images first, followed by a pooling layer to extract the most distinctive information.
    * The feature information is then used for classification tasks via a multi-layer perceptron (MLP).

### 2. CNN

#### 2.1 필요한 라이브러리 import - Importing Essential Libraries and Modules

In [ ]:
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split

from PIL import Image
import numpy as np

#### 2.2. Colab 환경에 데이터 저장 (kaggle API 사용) - Installing Kaggle API and Data Download

In [ ]:
!pip install kaggle --upgrade

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d samuelcortinhas/muffin-vs-chihuahua-image-classification
!unzip  muffin-vs-chihuahua-image-classification.zip

#### 2.3. 학습 관련 변수 사전 설정 - Setting up training-related variables

In [ ]:
lr = 0.001 # 학습률
batch_size = 100 # 한번에 얼만큼의 이미지를 학습시킬지
epochs = 10 # 데이터를 몇 번 반복하여 학습시킬지

#### 2.4 데이터 전처리 - Data PreProcessing

In [ ]:
# 주어진 이미지 데이터를 일괄적으로 300 * 300 의 크기로 정규화하고, 
# 텐서로 변환되도록 데이터 전처리 파이프라인을 정의합니다.
trans = transforms.Compose([transforms.Resize((300, 300)),
                           transforms.ToTensor()
                           ])

# 각 이미지 폴더에 위치한 이미지들을 정규화한 데이터셋으로 가져옵니다.
train_dataset = torchvision.datasets.ImageFolder(root= '/content/train',
                                                     transform = trans)

test_data = torchvision.datasets.ImageFolder(root= '/content/test',
                                                     transform = trans)

# train 데이터를 train / validation 데이터가 랜덤한 두 가지 값으로 나뉘도록 합니다.
val_size = 1000
train_size = len(train_dataset) - val_size
train_data,val_data = random_split(train_dataset,[train_size,val_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

# 각 학습용 데이터들을 사전 정의한 batch_size 단위로 처리할 수 있도록 데이터 로드를 생성합니다.
train_loader = DataLoader(dataset=train_data,
                          batch_size=batch_size,
                          shuffle=True)
val_loader = DataLoader(dataset=val_data,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_data,
                         batch_size=batch_size,
                         shuffle=False)

print(len(train_data), len(train_loader))
print(len(val_data), len(val_loader))

#### 2.5 CNN 모델 정의 - CNN Model Definition

In [ ]:
class Cnn(nn.Module):
    def __init__(self):
        super(Cnn,self).__init__()

        # 첫번째 레이어 정의
        self.layer1 = nn.Sequential(
            # 이미지를 입력값으로 받으므로, 최초 입력 채널의 크기는 3입니다.(RGB)
            # CNN에서 가장 기본적으로 사용되는 3*3 크기의 커널을 필터로 선택합니다.
            nn.Conv2d(3,16,kernel_size=3, padding=0,stride=2),
            # BatchNorm2d를 이용하여 학습 시 사용되는 batch를 정규화합니다.
            # 정규화 과정을 통해 학습을 더 안정적으로 진행할 수 있습니다.
            nn.BatchNorm2d(16),
            # 활성화 함수로는 Relu를 사용합니다.
            nn.ReLU(),
            # 2x2 Max 풀링을 수행하여 가장 큰 특징을 추출합니다.
            nn.MaxPool2d(2)
        )

        # 두번째 레이어 정의
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
            )
        
        # 세번째 레이어 정의
        self.layer3 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        
        # MLP 레이어 정의
        # 출력된 최종 레이어를 하나로 쭉 늘어뜨린 뒤, 10개의 출력층에 맵핑합니다.
        self.fc1 = nn.Linear(1296,10)
        # Dropout 레이어는 학습 중 일부 뉴런을 무작위로 비활성화하여 과적합을 방지합니다.
        self.dropout = nn.Dropout(0.5)
        # 위에서 10개로 줄어든 출력층을 최종 muffin/chihuahua 2개의 출력층으로 분류합니다.
        self.fc2 = nn.Linear(10,2)
        self.relu = nn.ReLU()

    # 상단에서 정의해준 각각의 레이어를 실행시키는 순방향전파 기능을 정의합니다.
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return out

#### 2.6 모델 학습 준비

In [ ]:
# 모델 생성 : 상단에서 정의한 모델 활용
model = Cnn().cuda()
# 모델 학습 모드 설정
model.train()
# 최적화 함수 설정 : Adam(Adaptive Moment Estimation)
# Adam? : Pytorch에 구현되어있는 경사하강법의 변형 중 하나
optimizer = optim.Adam(params = model.parameters(),lr=0.001)
# 손실 함수는 CrossEntropy함수를 사용합니다.
criterion = nn.CrossEntropyLoss()

#### 2.7. 모델 학습 수행

In [ ]:
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    # 학습 데이터 로더 train_loader에서 batch 데이터를 가져옵니다.
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        output = model(batch_images)
        loss = criterion(output, batch_labels)

        optimizer.zero_grad()   # 최적화 함수 가중치 초기화
        loss.backward()         # 역전파 수행하여 가중치 계산하고, 모델의 파라미터를 업데이트
        optimizer.step()        # 최적화 함수의 파라미터도 업데이트

        acc = ((output.argmax(dim=1) == batch_labels).float().mean())   # 예측값의 정확도를 계산합니다.
        epoch_accuracy += acc/len(train_loader)                         # 지금 수행중인 학습의 정확도와 손실을 누적합니다.
        epoch_loss += loss/len(train_loader)

    print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch+1, epoch_accuracy,epoch_loss))


    with torch.no_grad():       # validation data로 학습에 대한 검증을 수행합니다.
        epoch_val_accuracy=0
        epoch_val_loss =0
        for data, label in val_loader:
            val_output = model(data)
            val_loss = criterion(val_output,label)


            acc = ((val_output.argmax(dim=1) == label).float().mean())
            epoch_val_accuracy += acc/ len(val_loader)
            epoch_val_loss += val_loss/ len(val_loader)

        print('Epoch : {}, val_accuracy : {}, val_loss : {}'.format(epoch+1, epoch_val_accuracy,epoch_val_loss))

#### 2.8. 학습 결과 테스트

* 학습한 모델을 evaluation 모드로 설정한 후, 테스트용 사진 목록 중 랜덤한 사진을 뽑아 실제 라벨값과 모델이 예측한 라벨값을 비교합니다.

In [ ]:
import random
import matplotlib.pyplot as plt

# Set the model to evaluation mode
model.eval()

# Select a random image from the test dataset
random_image_idx = random.randint(0, len(test_data) - 1)
image, label = test_data[random_image_idx]

# Forward pass to get predictions
with torch.no_grad():
    image = image.unsqueeze(0).cuda()  # Add a batch dimension and move to GPU
    prediction = model(image)

predicted_class = prediction.argmax().item()

# Define a list of class labels (you can customize this based on your dataset)
class_labels =['chihuahua', 'muffin']

# Display the image and prediction
plt.imshow(image.squeeze(0).cpu().permute(1, 2, 0))
plt.title(f'Actual: {class_labels[label]}, Predicted: {class_labels[predicted_class]}')
plt.show()